<a href="https://colab.research.google.com/github/Kingfinalweapon/kings/blob/master/%E3%82%AA%E3%83%83%E3%82%BA_%E3%83%9C%E3%83%BC%E3%83%88%E7%9B%B4%E5%89%8D%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime as dt
import pandas as pd
from tqdm.notebook import tqdm
import datetime
import time
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
!pip install requests
from bs4 import BeautifulSoup
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
url = 'https://www.boatrace.jp/owpc/pc/race/racelist?rno=3&jcd=01&hd=20230329'
target_url = url.replace("/owpc/pc/race/racelist?", "/owpc/pc/race/odds3t?")
target_url

'https://www.boatrace.jp/owpc/pc/race/odds3t?rno=3&jcd=01&hd=20230329'

In [ ]:
# Webサイトからオッズデータを抽出する関数

ODDS_FILE_HEADER = "123,213,312,412,512,612,124,214,314,413,513,613,\
125,215,315,415,514,614,126,216,316,416,516,615,\
132,231,321,421,521,621,134,234,324,423,523,623,\
135,235,325,425,524,624,136,236,326,426,526,625,\
142,241,341,431,531,631,143,243,342,432,532,632,\
145,245,345,435,534,634,146,246,346,436,536,635,\
152,251,351,451,541,641,153,253,352,452,542,642,\
154,254,354,453,543,643,156,256,356,456,546,645,\
162,261,361,461,561,651,163,263,362,462,562,652,\
164,264,364,463,563,653,165,265,365,465,564,654"

time.sleep(1)
# BeautifulSoupにWebサイトのコンテンツを渡す
html = requests.get(target_url)
soup = BeautifulSoup(html.content, 'html.parser')
# Webサイトからコピーしたcss selectorを貼り付け　※3連単と2連単は同じ
target_table_selector = \
"body > main > div > div > div > div.contentsFrame1_inner"# > div:nth-child(6) > table"
# css selectorで指定したHTMLタグの中身を取得
odds_table = soup.select_one(target_table_selector)
odds_table = odds_table.find_all("div", attrs={"class": "table1"})[1].find_all("table")[0]
# オッズデータがあった場合
try:
    # tbodyを指定
    odds_table_elements = odds_table.select_one("tbody")
    # trを指定しリストとして格納
    row_list = odds_table_elements.select("tr")
    # オッズデータを格納する変数を定義
    csv_row = []
    # オッズデータを取得して変数に格納
    for row in row_list:
        for cell in row.select("td.oddsPoint"):
            csv_row.append(cell.get_text())
    temp = pd.DataFrame(csv_row).T

    temp.columns = ODDS_FILE_HEADER.split(",")

# オッズデータがなかった場合
except:
    print("no odds data")
    temp = pd.DataFrame([10000.] * 120).T
    temp.columns = ODDS_FILE_HEADER.split(",")


In [ ]:
temp

,123,213,312,412,512,612,124,214,314,413,...,364,463,563,653,165,265,365,465,564,654
0,11.9,25.2,64.4,110.7,557.2,47.0,23.4,38.1,66.4,134.9,...,105.4,196.0,1053,1133,110.6,223.2,241.2,328.1,887.2,832.4


In [ ]:
df_time = pd.read_html(target_url)[1]
df_time

,1,齋藤 真之,齋藤 真之.1,2,沢田 昭宏,沢田 昭宏.1,3,高野 心吾,高野 心吾.1,4,多田 有佑,多田 有佑.1,5,石渡 翔一郎,石渡 翔一郎.1,6,花田 和明,花田 和明.1
0,2,3,11.9,1,3,25.2,1,2,64.4,1,2,110.7,1,2,557.2,1,2,47.0
1,2,4,23.4,1,4,38.1,1,4,66.4,1,3,134.9,1,3,736.9,1,3,70.9
2,2,5,80.8,1,5,138.1,1,5,169.4,1,5,222.1,1,4,639.1,1,4,92.3
3,2,6,13.5,1,6,27.1,1,6,80.1,1,6,113.6,1,6,749.2,1,5,248.8
4,3,2,25.3,3,1,30.3,2,1,60.7,2,1,115.4,2,1,661.0,2,1,71.2
5,3,4,37.1,3,4,35.9,2,4,65.1,2,3,117.2,2,3,657.8,2,3,81.4
6,3,5,111.9,3,5,113.4,2,5,164.0,2,5,221.0,2,4,717.3,2,4,115.6
7,3,6,33.6,3,6,35.0,2,6,74.7,2,6,105.3,2,6,698.7,2,5,347.5
8,4,2,51.0,4,1,67.8,4,1,59.4,3,1,114.1,3,1,807.5,3,1,151.8
9,4,3,57.4,4,3,61.9,4,2,59.3,3,2,109.3,3,2,728.9,3,2,134.9
